In [ ]:
import numpy as np
import matplotlib.pyplot as plt

**Node class:** This class represents nodes in the decision tree. Nodes can either be internal nodes (decision nodes) or leaf nodes (terminal nodes). Internal nodes contain information about features and thresholds for splitting, as well as references to their left and right child nodes. Leaf nodes store the predicted value for a subset of data.

In [ ]:
class Node():

    def __init__(self, feature=None, threshold=None, left=None, right=None, gain=None, value=None):

        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.gain = gain
        self.value = value

**Class DecisionTree:**

- **split_data:** This method splits the dataset into two subsets based on a specified feature and threshold.

- **Gini:** This function calculates the Gini coefficient for a given set of labels. It is defined as:
  $$G(S) = 1 - \sum_{i=1}^{n} p_i^2,$$
  where \( p_i \) is the probability of each class label \( i \) in the dataset \( S \).

- **Information gain:** This function calculates the information gain by finding the difference between the Gini coefficient of the parent node and the weighted sum of the Gini coefficients of its child nodes. It uses the formula:
  $$IG(S, A) = G(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} G(S_v),$$
  where:
  - \( S \) is the dataset,
  - \( A \) is the attribute,
  - \( S_v \) are subsets of \( S \) for each value \( v \) of attribute \( A \).

- **best_split:** This function finds the best split for the dataset by iterating through all features and their unique values to calculate the information gain. It returns the index of the feature, threshold, and two resulting datasets that maximize the information gain.

- **calculate_leaf_value:** This function calculates the value for a leaf node. It finds the most common label in the given label set and assigns it as the value of the leaf node.

- **build_tree:** This function recursively builds the decision tree by finding the best split at each node based on information gain. It stops recursion when any of the criteria are met: minimum number of samples or maximum depth. It returns the root of the decision tree.

- **fit:** This function fits the decision tree to the training data. It constructs the dataset by combining features and labels, then builds the tree using the 'build_tree' function.

- **predict:** This function predicts labels for input data samples using the trained decision tree. It iterates through each sample and makes predictions by traversing the tree until reaching a leaf node.

- **make_prediction:** This function predicts the label for a single input sample by traversing the decision tree until reaching a leaf node. It returns the label assigned to the leaf node.


In [ ]:
class DecisionTree():

    def __init__(self, min_samples=2, max_depth=None):

        self.min_samples = min_samples
        self.max_depth = max_depth

    def split_data(self, dataset, feature, threshold):

        left_dataset = []
        right_dataset = []

        for row in dataset:
            if row[feature] <= threshold:
                left_dataset.append(row)
            else:
                right_dataset.append(row)

        left_dataset = np.array(left_dataset)
        right_dataset = np.array(right_dataset)
        return left_dataset, right_dataset

    def gini(self, y):
        gini = 1
        labels = np.unique(y)
        for label in labels:
            label_examples = y[y == label]
            pl = len(label_examples) / len(y)
            gini -= pl**2
        return gini


    def information_gain(self, parent, left, right):
        information_gain = 0
        parent_gini = self.gini(parent)
        weight_left = len(left) / len(parent)
        weight_right = len(right) / len(parent)
        gini_left, gini_right = self.gini(left), self.gini(right)
        weighted_gini = weight_left * gini_left + weight_right * gini_right
        information_gain = parent_gini - weighted_gini
        return information_gain


    def best_split(self, dataset, num_samples, num_features):

        best_split = {'gain': -1, 'feature': None, 'threshold': None}

        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]

            if isinstance(feature_values[0], float):
                thresholds = np.percentile(feature_values, np.linspace(0, 100, 100))
            else:
                thresholds = np.unique(feature_values)

            for threshold in thresholds:
                left_dataset, right_dataset = self.split_data(dataset, feature_index, threshold)
                if len(left_dataset) and len(right_dataset):
                    y, left_y, right_y = dataset[:, -1], left_dataset[:, -1], right_dataset[:, -1]
                    information_gain = self.information_gain(y, left_y, right_y)
                    if information_gain > best_split["gain"]:
                        best_split["feature"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_dataset"] = left_dataset
                        best_split["right_dataset"] = right_dataset
                        best_split["gain"] = information_gain
        return best_split


    def calculate_leaf_value(self, y):

        y = list(y)
        most_occuring_value = max(y, key=y.count)
        return most_occuring_value

    def build_tree(self, dataset, current_depth=0):

        X, y = dataset[:, :-1], dataset[:, -1]
        n_samples, n_features = X.shape

        if n_samples >= self.min_samples and (self.max_depth is None or current_depth < self.max_depth):
            best_split = self.best_split(dataset, n_samples, n_features)
            if best_split["gain"] > 0:
                left_dataset = best_split.get("left_dataset", None)
                right_dataset = best_split.get("right_dataset", None)

                if left_dataset is not None and right_dataset is not None:
                    left_node = self.build_tree(left_dataset, current_depth + 1)
                    right_node = self.build_tree(right_dataset, current_depth + 1)
                    return Node(best_split["feature"], best_split["threshold"],
                                left_node, right_node, best_split["gain"])

        leaf_value = self.calculate_leaf_value(y)
        return Node(value=leaf_value)

    def fit(self, X, y):

        dataset = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        self.root = self.build_tree(dataset)

    def predict(self, X):

        predictions = []
        for x in X:
            prediction = self.make_prediction(x, self.root)
            predictions.append(prediction)
        np.array(predictions)
        return predictions

    def make_prediction(self, x, node):

        if node.value != None:
            return node.value
        else:
            feature = x[node.feature]
            if feature <= node.threshold:
                return self.make_prediction(x, node.left)
            else:
                return self.make_prediction(x, node.right)

    def predict_proba(self, X):
        predictions = []
        for x in X:
            prediction = self.make_prediction(x, self.root)
            proba = [1 - prediction, prediction]
            predictions.append(proba)
        return np.array(predictions)

In [ ]:
def tree_plot(node, depth=0, xmin=-2, xmax=2):
    if node is None:
        return

    if node.feature is not None:
        x_center = (xmin + xmax) / 2
        y_center = depth

        x_left = xmin if node.left is None else (xmin + x_center) / 2
        x_right = xmax if node.right is None else (xmax + x_center) / 2

        offset = 0.3
        threshold_rounded = round(node.threshold, 3)
        plt.text(x_center, y_center, f"Feature {node.feature} \n <=  {threshold_rounded}", ha='center', va='center',
                 bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.2'))

        if node.left is not None:
            plt.arrow(x_center, y_center, x_left - x_center, -0.8, head_width=0.1, head_length=0.1, fc='black', ec='black')
            plt.text(x_left, y_center - 1, f"class:\n{node.left.value}", ha='center', va='center')
            tree_plot(node.left, depth - 1, xmin, x_center)

        if node.right is not None:
            plt.arrow(x_center, y_center, x_right - x_center, -0.8, head_width=0.1, head_length=0.1, fc='black', ec='black')
            plt.text(x_right, y_center - 1, f"class:\n{node.right.value}", ha='center', va='center')
            tree_plot(node.right, depth - 1, x_center, xmax)